In [ ]:
from google.colab import drive
drive.mount('/content/MyDrive')

Mounted at /content/MyDrive


In [ ]:
import csv
import requests

# Define your Musixmatch API key
MUSIXMATCH_API_KEY = ""
all_songs = []
# Function to search for tracks by artist
def search_songs(artist_name, page_size=50):
    url = f"https://api.musixmatch.com/ws/1.1/track.search"
    params = {
        "apikey": MUSIXMATCH_API_KEY,
        "q_artist": artist_name,
        "page_size": page_size,
        "s_track_rating": "desc"
    }
    res = requests.get(url, params=params)
    final_data = res.json()
    if "message" in final_data and "body" in final_data["message"]:
        songs = final_data["message"]["body"]["track_list"]
        return [(tr["track"]["track_name"], tr["track"]["artist_name"], tr["track"]["track_id"]) for tr in songs]
    else:
        print("Failed to fetch tracks.")
        return []

# Function to get lyrics for a track by track_id
def get_songs(track_id):
    url = f"https://api.musixmatch.com/ws/1.1/track.lyrics.get"
    params = {
        "apikey": MUSIXMATCH_API_KEY,
        "track_id": track_id
    }
    res = requests.get(url, params=params)
    final_data = res.json()
    if "message" in final_data and "body" in final_data["message"]:
        lyrics = final_data["message"]["body"]["lyrics"]["lyrics_body"]
        # Remove metafinal_data from the lyrics
        lyrics = lyrics.split("*******")[0]
        return lyrics.strip()
    else:
        print("Failed to fetch lyrics.")
        return ""

# Function to get lyrics for 50 songs of an artist
def get_artist_songs(artist_name):
    tracks = search_songs(artist_name)
    songs = []
    for track_name, artist, track_id in tracks:
        lyrics = get_songs(track_id)
        if lyrics:
            songs.append([track_name, artist, lyrics])
    return songs

# Function to write lyrics to CSV file
def write_to_csv(final_data, filename):
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Title", "Artist", "Lyrics"])
        writer.writerows(final_data)

# Main function
def main():
    artists = [
      "Eminem",
      "Pop Smoke",
      "Rihanna",
      "50 Cent",
      "Drake",
      "Future",
      "Jay Z",
      "Dr. Dre",
      "Travis Scott",
      "Megan Thee Stallion",

      "Cardi B",
      "Kanye West",
      "Nicki Minaj",
      "Lil Wayne"
    ]  # Add more artists as needed


    for artist in artists:
        print(f"Fetching lyrics for {artist}...")
        songs = get_artist_songs(artist)
        print(songs)
        all_songs.extend(songs)



if __name__ == "__main__":
    main()

Fetching lyrics for Eminem...
[['Love The Way You Lie', 'Eminem feat. Rihanna', 'Just gonna stand there and watch me burn?\nWell, that\'s alright because I like the way it hurts\nJust gonna stand there and hear me cry?\nWell, that\'s alright because I love the way you lie\nI love the way you lie\n\nI can\'t tell you what it really is\nI can only tell you what it feels like\nAnd right now, there\'s a steel knife in my windpipe\nI can\'t breathe, but I still fight while I can fight\nAs long as the wrong feels right, it\'s like I\'m in flight\nHigh off of love, drunk from my hate\nIt\'s like I\'m huffing paint, and I love her the more I suffer, I suffocate\nAnd right before I\'m about to drown, she resuscitates me\nShe fuckin\' hates me, and I love it\n\n"Wait! Where you going?" "I\'m leaving you"\n"No, you ain\'t, come back"\nWe\'re running right back, here we go again\nIt\'s so insane \'cause when it\'s going good, it\'s going great\nI\'m Superman with the wind at his back, she\'s Lois 

In [ ]:
def extend_songs():
  artists_extended = [
      "Snoop Dogg",
      "Lil Baby",
      "Post Malone",
      "Chance the Rapper",
      "Tyler, The Creator",
      "Juice WRLD",
      "Young Thug",
      "DaBaby",
      "Meek Mill",
      "G-Eazy",
      "Logic"
  ]


  for artist in artists_extended:
      print(f"Fetching lyrics for {artist}...")
      songs = get_artist_songs(artist)
      all_songs.extend(songs)
extend_songs()

Fetching lyrics for Snoop Dogg...
Fetching lyrics for Lil Baby...
Fetching lyrics for Post Malone...
Fetching lyrics for Chance the Rapper...
Fetching lyrics for Tyler, The Creator...
Fetching lyrics for Juice WRLD...
Fetching lyrics for Young Thug...
Fetching lyrics for DaBaby...
Fetching lyrics for Meek Mill...
Fetching lyrics for G-Eazy...
Fetching lyrics for Logic...


In [ ]:
write_to_csv(all_songs, "/content/MyDrive/MyDrive/NLP Project/lyrics.csv")

In [ ]:
#drop languages that aren't english
!pip install langdetect
from langdetect import detect
import pandas as pd

# Read the CSV file into a DataFrame
lyrics_df = pd.read_csv('lyrics.csv')

def detect_lyric(x):
    try:
        return detect(x)
    except:
        return

lyrics_df["language"] = lyrics_df["Lyrics"].apply(detect_lyric)
lyrics_df= lyrics_df[lyrics_df.language == "en"]

In [ ]:
lyrics = lyrics_df["Lyrics"].to_list()
lyrics = list(map(lambda x: x.replace("::", ":"),lyrics))
verses = [lyric.split(":") for lyric in lyrics]

import itertools

verses = list(itertools.chain(*verses))

def create_couplets(verses: list):
    couplets = []
    for i in range(1,len(verses)):
        couplet = verses[i-1] + "\n" + verses[i]
        couplets.append(couplet)

    return couplets


couplets = create_couplets(verses)
#display(couplets)

couplets_df = pd.DataFrame(couplets, columns=["couplets_g"])
print(couplets_df.head())

                                          couplets_g
0  Just gonna stand there and watch me burn?\nWel...
1  (Ugh, you're a monster)\n\nI can swallow a bot...
2  I'm friends with the monster that's under my b...
3  Yeah, It's been a ride...\nI guess I had to go...
4  I got a song filled with shit for the strong-w...
